# Notebook for building an XGBoost model for the Sendgrid sign up dataset
***
**Jake Mitchell Scott Schubert**

Initially using ______

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [6]:
train_path = 'data/signup_train_data.csv'

# Load the data into a DataFrame 
df = pd.read_csv(train_path, low_memory=False)
# Split into X and Y
dfLabels = df.pop("label")
#print (df.head())

#Temporarily parse non numerical data
df = df.drop(columns=['zip', 'city', 'website', 'state', 'country', 'registration_ip', 'company','multifactor_country_code','created_at','lead_source','marketing_channel','volume','user_persona','initial_package','employee_count','geolocation_notes','name_notes','ip_notes','community_notes','email_notes','activity_notes','fingerprint_notes'])
df.head()

,id,banned_ip,ip_count,is_authy_verified,banned_email,is_transactional,is_marketing,is_behavioral,is_oem,risk,...,ip_cluster,email_cluster,mfa_required,mfa_completed,whitelabel_required,whitelabel_completed,payment_required,payment_completed,profile_completed,email_completed
0,0,6,8,0,1,0,0,0,0,-100,...,False,False,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1,5,5,0,1,1,0,0,0,-90,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,1,0,0,1,0,0,0,-44,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,4,0,2,0,1,0,0,-72,...,False,False,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,4,0,1,0,0,1,0,0,0,-34,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# split data into train and test sets
seed = 7
test_size = 0.5
X_train, X_test, y_train, y_test = train_test_split(df, dfLabels, test_size=test_size, random_state=seed)


In [8]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [9]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 96.00%


/Users/schubydooo/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
pd.crosstab(predictions,y_test)

from xgboost import plot_importance
from matplotlib import pyplot
print(model.feature_importances_)
plot_importance(model)